<a href="https://colab.research.google.com/github/cwagner2325/WiseInbox_ML/blob/main/Wise_Inbox_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastapi
!pip install uvicorn
!pip install pickle5
!pip install pydantic
!pip install scikit-learn
!pip install requests
!pip install pypi-json
!pip install pyngrok
!pip install nest-asyncio

In [16]:
from fastapi import FastAPI
from pydantic import BaseModel
import pickle
import json
import uvicorn
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
import random
import os
from google.colab import userdata
from pyngrok import conf

In [17]:
app = FastAPI()

In [18]:
origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [19]:
class model_input(BaseModel):

    Email : str
    Sender: str
    Subject : str

In [20]:
# Example I found online. This guy used pickle to export a ML model as a file
# And then open the model and store it as a variable again using Pickle

# email_model = pickle.load(open('email_model.sav', 'rb'))

In [21]:
@app.post('/email_prediction')
def email_prediction(input_parameters : model_input):

    print("Email Prediction Reached")
    input_data = input_parameters.json()
    input_dictionary = json.loads(input_data)

    email = input_dictionary['Email']
    sender = input_dictionary['Sender']
    subject = input_dictionary['Subject']


    input_list = [email, sender, subject]

    prediction = random.randint(40, 100)
    print(prediction)

    # prediction = email_model.predict([input_list])

    return {"prediction": prediction}

In [22]:
os.environ["TOKEN"] = userdata.get("TOKEN")
ngrok.set_auth_token(os.environ["TOKEN"])

ngrok_tunnel = ngrok.connect(8000)

print('Public URL:', ngrok_tunnel.public_url)
print("Here is a test command to run in a terminal: ")
print('curl --insecure -X POST -H "Content-Type: application/json" -d \'{\n' +
      '\t"Email": "example@example.com",\n\t"Sender": "John Doe",\n\t"Subject": "Important News" \n}\' ' +
      ngrok_tunnel.public_url + "/email_prediction")

nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: https://d58a-35-221-41-212.ngrok-free.app
Here is a test command to run in a terminal: 
curl --insecure -X POST -H "Content-Type: application/json" -d '{
	"Email": "example@example.com",
	"Sender": "John Doe",
	"Subject": "Important News" 
}' https://d58a-35-221-41-212.ngrok-free.app/email_prediction


INFO:     Started server process [601]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Email Prediction Reached
97
INFO:     64.59.248.215:0 - "POST /email_prediction HTTP/1.1" 200 OK
Email Prediction Reached
68
INFO:     64.59.248.215:0 - "POST /email_prediction HTTP/1.1" 200 OK
Email Prediction Reached
70
INFO:     64.59.248.215:0 - "POST /email_prediction HTTP/1.1" 200 OK
Email Prediction Reached
60
INFO:     64.59.248.215:0 - "POST /email_prediction HTTP/1.1" 200 OK
Email Prediction Reached
82
INFO:     64.59.248.215:0 - "POST /email_prediction HTTP/1.1" 200 OK
Email Prediction Reached
48
INFO:     64.59.248.215:0 - "POST /email_prediction HTTP/1.1" 200 OK
Email Prediction Reached
55
INFO:     64.59.248.215:0 - "POST /email_prediction HTTP/1.1" 200 OK
Email Prediction Reached
52
INFO:     64.59.248.215:0 - "POST /email_prediction HTTP/1.1" 200 OK
Email Prediction Reached
55
INFO:     64.59.248.215:0 - "POST /email_prediction HTTP/1.1" 200 OK
Email Prediction Reached
81
INFO:     64.59.248.215:0 - "POST /email_prediction HTTP/1.1" 200 OK
Email Prediction Reached
82
IN

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [601]
